In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data

In [6]:
!pip install yfinance
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=fcffe42545352cc8e33636c6d569999671047a069e12ee6b47d2b85dbe6819ab
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [7]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
import tqdm
import yfinance as yf
from torch.utils.data import DataLoader
from torch.distributions import Categorical
import tqdm
from google.colab import runtime
# Finance Data
import pandas as pd
import pandas_ta as ta
from typing import List
from dataclasses import dataclass, field

import numpy as np
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset
from torch.utils.data import DataLoader


seaborn.set_context(context="talk")
%matplotlib inline

In [8]:
class TimeFeature:
    def __init__(self):
        pass

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        pass

    def __repr__(self):
        return self.__class__.__name__ + "()"


class SecondOfMinute(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.second / 59.0 - 0.5


class MinuteOfHour(TimeFeature):
    """Minute of hour encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.minute / 59.0 - 0.5


class HourOfDay(TimeFeature):
    """Hour of day encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.hour / 23.0 - 0.5


class DayOfWeek(TimeFeature):
    """Day of week encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return index.dayofweek / 6.0 - 0.5


class DayOfMonth(TimeFeature):
    """Day of month encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.day - 1) / 30.0 - 0.5


class DayOfYear(TimeFeature):
    """Day of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.dayofyear - 1) / 365.0 - 0.5


class MonthOfYear(TimeFeature):
    """Month of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.month - 1) / 11.0 - 0.5


class WeekOfYear(TimeFeature):
    """Week of year encoded as value between [-0.5, 0.5]"""

    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        return (index.isocalendar().week - 1) / 52.0 - 0.5


def time_features_from_frequency_str(freq_str: str) -> List[TimeFeature]:
    """
    Returns a list of time features that will be appropriate for the given frequency string.
    Parameters
    ----------
    freq_str
        Frequency string of the form [multiple][granularity] such as "12H", "5min", "1D" etc.
    """
    #print(f"freq_str: {freq_str}")

    features_by_offsets = {
        offsets.YearEnd: [],
        offsets.QuarterEnd: [MonthOfYear],
        offsets.MonthEnd: [MonthOfYear],
        offsets.Week: [DayOfMonth, WeekOfYear],
        offsets.Day: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.BusinessDay: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Hour: [HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Minute: [
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
        offsets.Second: [
            SecondOfMinute,
            MinuteOfHour,
            HourOfDay,
            DayOfWeek,
            DayOfMonth,
            DayOfYear,
        ],
    }

    offset = to_offset(freq_str)
    #print(offset)

    for offset_type, feature_classes in features_by_offsets.items():
        #print(f"offset_type: {offset_type}")
        if isinstance(offset, offset_type):
            #print(cls for cls in feature_classes)
            return [cls() for cls in feature_classes]

    supported_freq_msg = f"""
    Unsupported frequency {freq_str}
    The following frequencies are supported:
        Y   - yearly
            alias: A
        M   - monthly
        W   - weekly
        D   - daily
        B   - business days
        H   - hourly
        T   - minutely
            alias: min
        S   - secondly
    """
    raise RuntimeError(supported_freq_msg)


def time_features(dates, freq='h'):
    #print(f"Entered time_features!!")
    #print(dates)
    return np.vstack([feat(dates) for feat in time_features_from_frequency_str(freq)])


In [9]:
def extract_data(start_year, end_year=2023, ticker="^SPX"):
    data = yf.Ticker(ticker).history(period="max")
    data = data.dropna()

    def add_features(data):
      # Assuming your DataFrame is named 'data'
      data.ta.sma(close="Close", length=50, append=True)
      data.ta.sma(close="Close", length=200, append=True)
      #data.ta.ichimoku(close="Close", append=True)
      #data.ta.macd(close="Close", append=True)
      data.ta.rsi(close="Close", append=True)
      data.ta.bbands(close="Close", append=True)
      data.ta.macd(close="Close", append=True)
      data.ta.ichimoku(close="Close", append=True)
      data.ta.smi(close="Close", append=True)
      data.ta.willr(close="Close", low="Low", high="High", append=True)
      data.ta.stoch(close="Close", low="Low", high="High", append=True)
      data.ta.fisher(low="Low", high="High", append=True)
      data.ta.atr(low="Low", high="High", close="Close", append=True)
      #data.ta.cdl_pattern(name=['eveningstar', '3whitesoldiers', 'morningstar', '3blackcrows', '3linestrike'])
      data.ta.obv(volume="Volume", close="Close", append=True)
      data.ta.zscore(close="Close", append=True)
      data.ta.entropy(close="Close", append=True)
      return data

    data = add_features(data)
    ## Columns to Drop
    drop = ['Volume', 'Dividends', 'Stock Splits']
    data = data.drop(drop, axis=1)
    data = data.dropna()
    start_year = data.index[0].year if start_year is None else start_year
    data = data[data.index.year >= start_year-1] if start_year is not None else data
    data = data[data.index.year <= end_year] if end_year is not None else data
    #print(f"start year: {start_year}")
    #print(data)
    return data

df = extract_data(1990)
#df = df['Close']
df

,Open,High,Low,Close,SMA_50,SMA_200,RSI_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,...,SMIo_5_20_5,WILLR_14,STOCHk_14_3_3,STOCHd_14_3_3,FISHERT_9_1,FISHERTs_9_1,ATRr_14,OBV,ZS_30,ENTP_10
Date,,,,,,,,,,,,,,,,,,,,,
1989-01-03 00:00:00-05:00,277.720001,277.720001,273.809998,275.309998,274.762200,268.111900,48.838065,274.613084,277.267993,279.922903,...,-0.057734,-77.409688,54.631409,58.669202,0.388937,0.797978,2.233492,2.680319e+10,0.229301,3.325722
1989-01-04 00:00:00-05:00,275.309998,279.750000,275.309998,279.429993,274.677599,268.165350,58.288271,274.697225,277.787994,280.878764,...,-0.011009,-15.361706,54.945716,57.522858,0.248036,0.388937,2.391100,2.695289e+10,1.197195,3.325502
1989-01-05 00:00:00-05:00,279.429993,281.510010,279.429993,280.010010,274.632200,268.221200,59.424554,274.949366,278.373999,281.798632,...,0.014795,-19.480489,62.582706,57.386610,0.507648,0.248036,2.368880,2.712693e+10,1.312146,3.326466
1989-01-06 00:00:00-05:00,280.010010,282.059998,280.010010,280.670013,274.598000,268.280000,60.735306,274.775049,278.628003,282.480957,...,0.030888,-16.848292,82.769838,66.766086,0.908969,0.507648,2.346102,2.728826e+10,1.453896,3.327660
1989-01-09 00:00:00-05:00,280.670013,281.890015,280.320007,280.980011,274.590000,268.368150,61.366579,275.167730,279.280005,283.392280,...,0.037330,-13.090746,83.526824,76.293122,1.352505,0.908969,2.290666,2.745144e+10,1.477710,3.329150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,4753.919922,4772.939941,4736.770020,4754.629883,4468.218584,4335.861046,71.069402,4694.534676,4741.732031,4788.929386,...,-0.054860,-10.098875,80.663261,83.908053,2.832599,2.986470,41.184150,1.379242e+12,1.638294,3.350524
2023-12-26 00:00:00-05:00,4758.859863,4784.720215,4758.450195,4774.750000,4477.157988,4340.426846,72.704374,4694.610700,4748.570020,4802.529339,...,-0.035086,-4.185293,90.737754,85.128169,2.967104,2.832599,40.391734,1.381756e+12,1.736890,3.350531
2023-12-27 00:00:00-05:00,4773.450195,4785.390137,4768.899902,4781.580078,4485.316992,4345.055946,73.256889,4692.545294,4751.212012,4809.878729,...,-0.019366,-1.730508,94.661774,88.687597,3.241961,2.967104,38.684484,1.384504e+12,1.720615,3.349759


In [10]:
import pandas as pd
import numpy as np

def datetime_to_one_hot(datetime_values):
    # Convert input to a DatetimeIndex if it's not already
    if isinstance(datetime_values, pd.Series):
        datetime_values = pd.to_datetime(datetime_values)
    elif isinstance(datetime_values, pd.Index):
        datetime_values = pd.DatetimeIndex(datetime_values)
    else:
        raise ValueError("Input should be a Pandas Series or DatetimeIndex.")

    # Extract the day of the week (0=Monday, 6=Sunday)
    day_of_week = datetime_values.dayofweek

    # Extract the day of the month (1-31)
    day_of_month = datetime_values.day

    # Extract the week of the month (1-5)
    week_of_month = (datetime_values.day - 1) // 7 + 1

    # Extract the month of the year (1-12)
    month_of_year = datetime_values.month

    # One-hot encode each component
    one_hot_day_of_week = np.eye(7)[day_of_week]
    one_hot_day_of_month = np.eye(31)[day_of_month - 1]  # day_of_month ranges from 1-31, so subtract 1 for 0-based indexing
    one_hot_week_of_month = np.eye(5)[week_of_month - 1]  # week_of_month ranges from 1-5, so subtract 1 for 0-based indexing
    one_hot_month_of_year = np.eye(12)[month_of_year - 1]  # month_of_year ranges from 1-12, so subtract 1 for 0-based indexing

    # Concatenate all one-hot vectors along the second axis
    one_hot_combined = np.concatenate([one_hot_day_of_week, one_hot_day_of_month, one_hot_week_of_month, one_hot_month_of_year], axis=1)

    return one_hot_combined


In [7]:
cols = list(df.columns)
raw_data = df[:3]
raw_data

,Open,High,Low,Close,SMA_50,SMA_200,RSI_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,...,SMIo_5_20_5,WILLR_14,STOCHk_14_3_3,STOCHd_14_3_3,FISHERT_9_1,FISHERTs_9_1,ATRr_14,OBV,ZS_30,ENTP_10
Date,,,,,,,,,,,,,,,,,,,,,
1989-01-03 00:00:00-05:00,277.720001,277.720001,273.809998,275.309998,274.762200,268.11190,48.838065,274.613084,277.267993,279.922903,...,-0.057734,-77.409688,54.631409,58.669202,0.388937,0.797978,2.233492,2.680319e+10,0.229301,3.325722
1989-01-04 00:00:00-05:00,275.309998,279.750000,275.309998,279.429993,274.677599,268.16535,58.288271,274.697225,277.787994,280.878764,...,-0.011009,-15.361706,54.945716,57.522858,0.248036,0.388937,2.391100,2.695289e+10,1.197195,3.325502
1989-01-05 00:00:00-05:00,279.429993,281.510010,279.429993,280.010010,274.632200,268.22120,59.424554,274.949366,278.373999,281.798632,...,0.014795,-19.480489,62.582706,57.386610,0.507648,0.248036,2.368880,2.712693e+10,1.312146,3.326466


In [11]:
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

class StockDataset(Dataset):
    def __init__(self, tickers, flag='train', size=None, one_hot_datetime=False,
                 features='S', target='Close', scale=True, timeenc=0, freq='d', batch_size=5,
                 data_start_year=1990, data_end_year=2023):
        if size is None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]

        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]
        self.data_start_year = data_start_year
        self.data_end_year = data_end_year

        self.features = features
        self.target = target
        self.scale = scale
        self.one_hot_datetime = one_hot_datetime
        self.timeenc = timeenc
        self.freq = freq
        self.scaler = StandardScaler()
        self.tickers = tickers.split()
        self.ticker_database = {}
        self.batch_size = batch_size
        self.min_year = 0
        self.__read_data__()

    def __len__(self):
        return len(self.years) - 1

    def get_min_year(self):
        min_year = 0
        for ticker in self.tickers:
            self.ticker_database[ticker] = extract_data(start_year=None, ticker=ticker)
            min_year = max(min_year, self.ticker_database[ticker].index.year.min())
        return min_year + 2

    def __read_data__(self):
        print(f"Loading following tickers: {self.tickers}\n")

        self.min_year = max(self.get_min_year(), self.data_start_year)
        print(f"Dataset Start Year: {self.min_year} | End Year: {self.data_end_year}")
        for ticker in self.tickers:
            self.ticker_database[ticker] = extract_data(start_year=self.min_year, end_year=self.data_end_year, ticker=ticker)
            self.ticker_database[ticker]['year'] = self.ticker_database[ticker].index.year

        self.years = self.ticker_database[self.tickers[0]]['year'].unique()
        print(f"years: {self.years}")
        self.data_by_year = {year: {ticker: self.ticker_database[ticker][self.ticker_database[ticker]['year'] == year] for ticker in self.tickers} for year in self.years}
        print(f"Normalizing Data: {self.scale}")
        self.data_len = len(self.years)
        print(f"DateTime is one-hot: {self.one_hot_datetime}")

    def __getitem__(self, idx):
        """
        Final item form is a list containing [seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates]
        batches_x = batches[0]
        batches_y = batches[1]
        batches_x_mark = batches[2]
        batches_y_mark = batches[3]
        batches_x_dates = batches[4]
        batches_y_dates = batches[5]

        Each batches, contain batch data of size (batch_size, seq_len, num_features)
        For example, if seq_len = 24, batch_size = 5, num_features = 32,
        Each item of batches_x is a tensor of (5, 24, 32)
        """
        year = self.min_year + idx if idx >= 0 else self.max_year + idx + 1
        #print(f"Stock Dataset Yeaer: {year}")
        raw_datas = []

        for ticker in self.tickers:
            if year - 1 in self.data_by_year:
                prev_year_data = self.data_by_year[year - 1][ticker].tail(self.seq_len + self.pred_len - 1)
                #print(f"Prev year data for {ticker} in {year}: {prev_year_data.index}")
            else:
                print(f"Previous Year Data is Insufficient, Year: {year-1}, Ticker: {ticker}")
                prev_year_data = pd.DataFrame()

            ticker_data = pd.concat([prev_year_data, self.data_by_year[year][ticker]])
            #print(f"Ticker data for {ticker} in {year}: {ticker_data.index}")
            ticker_data['date'] = ticker_data.index
            raw_datas.append(ticker_data)
            """print(f"Ticker: {ticker}")
            print(len(ticker_data))
            print(ticker_data)"""

        seq_x = []
        seq_y = []
        seq_x_mark = []
        seq_y_mark = []
        seq_x_dates = []
        seq_y_dates = []

        for item in raw_datas:
            x, y, x_mark, y_mark, x_dates, y_dates = self.make_data(item)
            seq_x.append(x)
            seq_y.append(y)
            seq_x_mark.append(x_mark)
            seq_y_mark.append(y_mark)
            seq_x_dates.append(x_dates)
            seq_y_dates.append(y_dates)

        # Combine all tickers into a single batch and slice them into mini-batches
        #print(len(seq_x))
        return self.create_batches(seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates), year

    def make_data(self, raw_data):
        cols = list(raw_data.columns)
        cols.remove(self.target)
        cols.remove('date')
        cols.remove('year')
        raw_data = raw_data[['date'] + cols + [self.target]]

        if self.features == 'MS':
            cols_data = raw_data.columns[1:]
            #print(f"cols_data: {cols_data}")
            data = raw_data[cols_data]
        elif self.features == 'S':
            data = raw_data['Close']

        if self.scale:
            # data = (data - data.min()) / (data.max() - data.min())
            data = self.scaler.fit_transform(data.values)

        if self.one_hot_datetime:
          #print(type(pd.to_datetime(raw_data['date'].values)))
          data_stamp = datetime_to_one_hot(pd.to_datetime(raw_data['date'].values))
          #print(one_hot_data_stamp.shape)
        else:
          data_stamp = time_features(pd.to_datetime(raw_data['date'].values), freq=self.freq)
          data_stamp = data_stamp.transpose(1, 0)

        """print("----Yearly data----")
        print(data[0].shape)
        print(len(data))
        print(data)"""

        seq_x = []
        seq_y = []
        seq_x_mark = []
        seq_y_mark = []
        x_dates = []
        y_dates = []

        for i in range(len(data) - self.pred_len - self.seq_len + 1):
            s_begin = i
            s_end = s_begin + self.seq_len
            r_begin = s_end - self.label_len
            r_end = r_begin + self.label_len + self.pred_len

            seq_x.append(data[s_begin:s_end])
            seq_y.append(data[r_begin:r_end])
            seq_x_mark.append(data_stamp[s_begin:s_end])
            seq_y_mark.append(data_stamp[r_begin:r_end])
            x_dates.append(raw_data['date'][s_begin:s_end])
            y_dates.append(raw_data['date'][r_begin:r_end])
            """print(f"data[s_begin:s_end]: {data[s_begin:s_end]}")
            print(f"data[r_begin:r_end]: {data[r_begin:r_end]}")
            print(f"raw_data['date'][s_begin:s_end]: {raw_data['date'][s_begin:s_end]}")
            print(f"raw_data['date'][r_begin:r_end]: {raw_data['date'][r_begin:r_end]}")
            print("=======================================")"""

        return torch.tensor(np.array(seq_x)), torch.tensor(np.array(seq_y)), torch.tensor(np.array(seq_x_mark)), torch.tensor(np.array(seq_y_mark)), x_dates, y_dates

    def create_batches(self, seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates, dates=None):
        batches_x = []
        batches_y = []
        batches_x_mark = []
        batches_y_mark = []
        batches_x_dates = []
        batches_y_dates = []
        batch_size = self.batch_size
        #print(dates[-1])
        for x, y, x_mark, y_mark, x_dates, y_dates in zip(seq_x, seq_y, seq_x_mark, seq_y_mark, seq_x_dates, seq_y_dates):
            #print(x.shape)
            for i in range(0, x.shape[0], batch_size):
                batches_x.append(x[i:i + batch_size])
                batches_y.append(y[i:i + batch_size])
                #print(f"x[i:i + batch_size]: {x[i:i + batch_size].shape}")
                #print(f"y[i:i + batch_size]: {y[i:i + batch_size].shape}")
                batches_x_mark.append(x_mark[i:i + batch_size])
                batches_y_mark.append(y_mark[i:i + batch_size])
                batches_x_dates.append(x_dates[i:i + batch_size])
                batches_y_dates.append(y_dates[i:i + batch_size])
                """print(f"x[i:i + batch_size]: {x[i:i + batch_size]}")
                print(f"y[i:i + batch_size]: {y[i:i + batch_size]}")
                print(f"x_dates[i:i + batch_size]: {x_dates[i:i + batch_size]}")
                print(f"y_dates[i:i + batch_size]: {y_dates[i:i + batch_size]}")
                print(f"------------------------------------------------------")"""

        return batches_x, batches_y, batches_x_mark, batches_y_mark, batches_x_dates, batches_y_dates

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

In [9]:
dataset = StockDataset(tickers='^SPX',timeenc=1, freq='d', size=[36, 18, 1], features='MS')

Loading following tickers: ['^SPX']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]
Normalizing Data: True
DateTime is one-hot: False


In [10]:
batches, year = dataset[0]

# Model

In [12]:
!pip install einops

In [50]:
import torch
import torch.nn as nn
import numpy as np
import math

import torch.fft as fft
from einops import rearrange, reduce, repeat
import einops


class SparseDispatcher(object):
    def __init__(self, num_experts, gates):
        """Create a SparseDispatcher."""

        self._gates = gates
        self._num_experts = num_experts

        # sort experts
        sorted_experts, index_sorted_experts = torch.nonzero(gates).sort(0)
        _, self._expert_index = sorted_experts.split(1, dim=1)
        # get according batch index for each expert
        self._batch_index = torch.nonzero(gates)[index_sorted_experts[:, 1], 0]
        self._part_sizes = (gates > 0).sum(0).tolist()
        gates_exp = gates[self._batch_index.flatten()]
        self._nonzero_gates = torch.gather(gates_exp, 1, self._expert_index)

    def dispatch(self, inp):
        # assigns samples to experts whose gate is nonzero
        # expand according to batch index so we can just split by _part_sizes
        inp_exp = inp[self._batch_index].squeeze(1)
        return torch.split(inp_exp, self._part_sizes, dim=0)

    def combine(self, expert_out, multiply_by_gates=True):
        # apply exp to expert outputs, so we are not longer in log space
        stitched = torch.cat(expert_out, 0).exp()
        if multiply_by_gates:
            stitched = torch.einsum("ijkh,ik -> ijkh", stitched, self._nonzero_gates)
        zeros = torch.zeros(self._gates.size(0), expert_out[-1].size(1), expert_out[-1].size(2), expert_out[-1].size(3),
                            requires_grad=True, device=stitched.device)
        # combine samples that have been processed by the same k experts
        combined = zeros.index_add(0, self._batch_index, stitched.float())
        # add eps to all zero values in order to avoid nans when going back to log space
        combined[combined == 0] = np.finfo(float).eps
        # back to log space
        return combined.log()
    def expert_to_gates(self):
        # split nonzero gates for each expert
        return torch.split(self._nonzero_gates, self._part_sizes, dim=0)


class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc = nn.Conv2d(in_channels=input_size,
                             out_channels=output_size,
                             kernel_size=(1, 1),
                             bias=True)

    def forward(self, x):
        out = self.fc(x)
        return out



class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """

    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, self.kernel_size - 1 - math.floor((self.kernel_size - 1) // 2), 1)
        end = x[:, -1:, :].repeat(1, math.floor((self.kernel_size - 1) // 2), 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """

    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean


class series_decomp_multi(nn.Module):
    """
    Series decomposition block
    """

    def __init__(self, kernel_size):
        super(series_decomp_multi, self).__init__()
        self.moving_avg = [moving_avg(kernel, stride=1) for kernel in kernel_size]
        self.layer = torch.nn.Linear(1, len(kernel_size))

    def forward(self, x):
        moving_mean = []
        for func in self.moving_avg:
            moving_avg = func(x)
            moving_mean.append(moving_avg.unsqueeze(-1))
        moving_mean = torch.cat(moving_mean, dim=-1)
        moving_mean = torch.sum(moving_mean * nn.Softmax(-1)(self.layer(x.unsqueeze(-1))), dim=-1)
        res = x - moving_mean
        return res, moving_mean


class FourierLayer(nn.Module):

    def __init__(self, pred_len, k=None, low_freq=1, output_attention=False):
        super().__init__()
        # self.d_model = d_model
        self.pred_len = pred_len
        self.k = k
        self.low_freq = low_freq
        self.output_attention = output_attention

    def forward(self, x):
        """x: (b, t, d)"""

        if self.output_attention:
            return self.dft_forward(x)

        b, t, d = x.shape
        x_freq = fft.rfft(x, dim=1)

        if t % 2 == 0:
            x_freq = x_freq[:, self.low_freq:-1]
            f = fft.rfftfreq(t)[self.low_freq:-1]
        else:
            x_freq = x_freq[:, self.low_freq:]
            f = fft.rfftfreq(t)[self.low_freq:]

        x_freq, index_tuple = self.topk_freq(x_freq)
        f = repeat(f, 'f -> b f d', b=x_freq.size(0), d=x_freq.size(2))
        f = f.to(x_freq.device)
        f = rearrange(f[index_tuple], 'b f d -> b f () d').to(x_freq.device)

        return self.extrapolate(x_freq, f, t), None

    def extrapolate(self, x_freq, f, t):
        x_freq = torch.cat([x_freq, x_freq.conj()], dim=1)
        f = torch.cat([f, -f], dim=1)
        t_val = rearrange(torch.arange(t + self.pred_len, dtype=torch.float),
                          't -> () () t ()').to(x_freq.device)

        amp = rearrange(x_freq.abs() / t, 'b f d -> b f () d')
        phase = rearrange(x_freq.angle(), 'b f d -> b f () d')

        x_time = amp * torch.cos(2 * math.pi * f * t_val + phase)
        return einops.reduce(x_time, 'b f t d -> b t d', 'sum')

    def topk_freq(self, x_freq):
        values, indices = torch.topk(x_freq.abs(), self.k, dim=1, largest=True, sorted=True)
        mesh_a, mesh_b = torch.meshgrid(torch.arange(x_freq.size(0)), torch.arange(x_freq.size(2)))
        index_tuple = (mesh_a.unsqueeze(1), indices, mesh_b.unsqueeze(1))
        x_freq = x_freq[index_tuple]

        return x_freq, index_tuple

    def dft_forward(self, x):
        T = x.size(1)

        dft_mat = fft.fft(torch.eye(T))
        i, j = torch.meshgrid(torch.arange(self.pred_len + T), torch.arange(T))
        omega = np.exp(2 * math.pi * 1j / T)
        idft_mat = (np.power(omega, i * j) / T).cfloat()

        x_freq = torch.einsum('ft,btd->bfd', [dft_mat, x.cfloat()])

        if T % 2 == 0:
            x_freq = x_freq[:, self.low_freq:T // 2]
        else:
            x_freq = x_freq[:, self.low_freq:T // 2 + 1]

        _, indices = torch.topk(x_freq.abs(), self.k, dim=1, largest=True, sorted=True)
        indices = indices + self.low_freq
        indices = torch.cat([indices, -indices], dim=1)

        dft_mat = repeat(dft_mat, 'f t -> b f t d', b=x.shape[0], d=x.shape[-1])
        idft_mat = repeat(idft_mat, 't f -> b t f d', b=x.shape[0], d=x.shape[-1])

        mesh_a, mesh_b = torch.meshgrid(torch.arange(x.size(0)), torch.arange(x.size(2)))

        dft_mask = torch.zeros_like(dft_mat)
        dft_mask[mesh_a, indices, :, mesh_b] = 1
        dft_mat = dft_mat * dft_mask

        idft_mask = torch.zeros_like(idft_mat)
        idft_mask[mesh_a, :, indices, mesh_b] = 1
        idft_mat = idft_mat * idft_mask

        attn = torch.einsum('bofd,bftd->botd', [idft_mat, dft_mat]).real
        return torch.einsum('botd,btd->bod', [attn, x]), rearrange(attn, 'b o t d -> b d o t')

In [51]:
import torch
import torch.nn as nn
import numpy as np
import math

class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, n_position=1024):
        super(PositionalEmbedding, self).__init__()

        # Not a parameter
        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_position, d_model))

    def _get_sinusoid_encoding_table(self, n_position, d_model):
        ''' Sinusoid position encoding table '''
        def get_position_angle_vec(position):
            return [position / np.power(10000, 2 * (hid_j // 2) / d_model) for hid_j in range(d_model)]

        sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)])
        sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
        sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

        return torch.FloatTensor(sinusoid_table).unsqueeze(0)

    def forward(self, x):

        return self.pos_table[:, :x.size(1)].clone().detach()


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class FixedEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(FixedEmbedding, self).__init__()

        w = torch.zeros(c_in, d_model).float()
        w.require_grad = False

        position = torch.arange(0, c_in).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        w[:, 0::2] = torch.sin(position * div_term)
        w[:, 1::2] = torch.cos(position * div_term)

        self.emb = nn.Embedding(c_in, d_model)
        self.emb.weight = nn.Parameter(w, requires_grad=False)

    def forward(self, x):
        return self.emb(x).detach()


class TemporalEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='fixed', freq='h'):
        super(TemporalEmbedding, self).__init__()

        minute_size = 4
        hour_size = 24
        weekday_size = 7
        day_size = 32
        month_size = 13

        Embed = FixedEmbedding if embed_type == 'fixed' else nn.Embedding
        if freq == 't':
            self.minute_embed = Embed(minute_size, d_model)
        self.hour_embed = Embed(hour_size, d_model)
        self.weekday_embed = Embed(weekday_size, d_model)
        self.day_embed = Embed(day_size, d_model)
        self.month_embed = Embed(month_size, d_model)

    def forward(self, x):
        x = x.long()

        minute_x = self.minute_embed(x[:, :, 4]) if hasattr(self, 'minute_embed') else 0.
        hour_x = self.hour_embed(x[:, :, 3])
        weekday_x = self.weekday_embed(x[:, :, 2])
        day_x = self.day_embed(x[:, :, 1])
        month_x = self.month_embed(x[:, :, 0])

        return hour_x + weekday_x + day_x + month_x + minute_x


class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='h'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h': 4, 't': 5, 's': 6, 'm': 1, 'a': 1, 'w': 2, 'd': 3, 'b': 3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model, bias=False)

    def forward(self, x):
        return self.embed(x)


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, embed_type='fixed', freq='h', dropout=0.1):
        super(DataEmbedding, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.temporal_embedding = TemporalEmbedding(d_model=d_model, embed_type=embed_type,
                                                    freq=freq) if embed_type != 'timeF' else TimeFeatureEmbedding(
            d_model=d_model, embed_type=embed_type, freq=freq)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark):
        x = self.value_embedding(x) + self.temporal_embedding(x_mark) + self.position_embedding(x)
        return self.dropout(x)


class DataEmbedding_wo_temp(nn.Module):
    def __init__(self, c_in, d_model, dropout=0.1):
        super(DataEmbedding_wo_temp, self).__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, x_mark=None):
        x = self.value_embedding(x) + self.position_embedding(x)

        return self.dropout(x)




def PositionalEncoding(q_len, d_model, normalize=True):
    pe = torch.zeros(q_len, d_model)
    position = torch.arange(0, q_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    if normalize:
        pe = pe - pe.mean()
        pe = pe / (pe.std() * 10)
    return pe

SinCosPosEncoding = PositionalEncoding

def Coord2dPosEncoding(q_len, d_model, exponential=False, normalize=True, eps=1e-3, verbose=False):
    x = .5 if exponential else 1
    i = 0
    for i in range(100):
        cpe = 2 * (torch.linspace(0, 1, q_len).reshape(-1, 1) ** x) * (torch.linspace(0, 1, d_model).reshape(1, -1) ** x) - 1
        pv(f'{i:4.0f}  {x:5.3f}  {cpe.mean():+6.3f}', verbose)
        if abs(cpe.mean()) <= eps: break
        elif cpe.mean() > eps: x += .001
        else: x -= .001
        i += 1
    if normalize:
        cpe = cpe - cpe.mean()
        cpe = cpe / (cpe.std() * 10)
    return cpe

def Coord1dPosEncoding(q_len, exponential=False, normalize=True):
    cpe = (2 * (torch.linspace(0, 1, q_len).reshape(-1, 1)**(.5 if exponential else 1)) - 1)
    if normalize:
        cpe = cpe - cpe.mean()
        cpe = cpe / (cpe.std() * 10)
    return cpe

def positional_encoding(pe, learn_pe, q_len, d_model):
    # Positional encoding
    if pe == None:
        W_pos = torch.empty((q_len, d_model)) # pe = None and learn_pe = False can be used to measure impact of pe
        nn.init.uniform_(W_pos, -0.02, 0.02)
        learn_pe = False
    elif pe == 'zero':
        W_pos = torch.empty((q_len, 1))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == 'zeros':
        W_pos = torch.empty((q_len, d_model))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == 'normal' or pe == 'gauss':
        W_pos = torch.zeros((q_len, 1))
        torch.nn.init.normal_(W_pos, mean=0.0, std=0.1)
    elif pe == 'uniform':
        W_pos = torch.zeros((q_len, 1))
        nn.init.uniform_(W_pos, a=0.0, b=0.1)
    elif pe == 'lin1d': W_pos = Coord1dPosEncoding(q_len, exponential=False, normalize=True)
    elif pe == 'exp1d': W_pos = Coord1dPosEncoding(q_len, exponential=True, normalize=True)
    elif pe == 'lin2d': W_pos = Coord2dPosEncoding(q_len, d_model, exponential=False, normalize=True)
    elif pe == 'exp2d': W_pos = Coord2dPosEncoding(q_len, d_model, exponential=True, normalize=True)
    elif pe == 'sincos': W_pos = PositionalEncoding(q_len, d_model, normalize=True)
    else: raise ValueError(f"{pe} is not a valid pe (positional encoder. Available types: 'gauss'=='normal', \
        'zeros', 'zero', uniform', 'lin1d', 'exp1d', 'lin2d', 'exp2d', 'sincos', None.)")
    return nn.Parameter(W_pos, requires_grad=learn_pe)

In [98]:
import math
import torch
import torch.nn as nn
from torch.nn import init
import time
import torch.nn.functional as F

class Transformer_Layer(nn.Module):
    def __init__(self, device, d_model, d_ff, num_nodes, patch_nums, patch_size, dynamic, factorized, layer_number, batch_norm):
        super(Transformer_Layer, self).__init__()
        self.device = device
        self.d_model = d_model
        self.num_nodes = num_nodes
        self.dynamic = dynamic
        self.patch_nums = patch_nums
        self.patch_size = patch_size
        self.layer_number = layer_number
        self.batch_norm = batch_norm


        ##intra_patch_attention
        self.intra_embeddings = nn.Parameter(torch.rand(self.patch_nums, 1, 1, self.num_nodes, 16),
                                                requires_grad=True)
        self.embeddings_generator = nn.ModuleList([nn.Sequential(*[
            nn.Linear(16, self.d_model)]) for _ in range(self.patch_nums)])
        self.intra_d_model = self.d_model
        self.intra_patch_attention = Intra_Patch_Attention(self.intra_d_model, factorized=factorized)
        self.weights_generator_distinct = WeightGenerator(self.intra_d_model, self.intra_d_model, mem_dim=16, num_nodes=num_nodes,
                                                          factorized=factorized, number_of_weights=2)
        self.weights_generator_shared = WeightGenerator(self.intra_d_model, self.intra_d_model, mem_dim=None, num_nodes=num_nodes,
                                                        factorized=False, number_of_weights=2)
        self.intra_Linear = nn.Linear(self.patch_nums, self.patch_nums*self.patch_size)



        ##inter_patch_attention
        self.stride = patch_size
        # patch_num = int((context_window - cut_size) / self.stride + 1)

        self.inter_d_model = self.d_model * self.patch_size
        ##inter_embedding
        self.emb_linear = nn.Linear(self.inter_d_model, self.inter_d_model)
        # Positional encoding
        self.W_pos = positional_encoding(pe='zeros', learn_pe=True, q_len=self.patch_nums, d_model=self.inter_d_model)
        n_heads = self.d_model
        d_k = self.inter_d_model // n_heads
        d_v = self.inter_d_model // n_heads
        self.inter_patch_attention = Inter_Patch_Attention(self.inter_d_model, self.inter_d_model, n_heads, d_k, d_v, attn_dropout=0,
                                          proj_dropout=0.1, res_attention=False)


        ##Normalization
        self.norm_attn = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(self.d_model), Transpose(1,2))
        self.norm_ffn = nn.Sequential(Transpose(1,2), nn.BatchNorm1d(self.d_model), Transpose(1,2))

        ##FFN
        self.d_ff = d_ff
        self.dropout = nn.Dropout(0.1)
        self.ff = nn.Sequential(nn.Linear(self.d_model, self.d_ff, bias=True),
                                nn.GELU(),
                                nn.Dropout(0.2),
                                nn.Linear(self.d_ff, self.d_model, bias=True))

    def forward(self, x):

        print(f"x shape: {x.shape}")
        new_x  = x
        batch_size = x.size(0)
        intra_out_concat = None

        weights_shared, biases_shared = self.weights_generator_shared()
        weights_distinct, biases_distinct = self.weights_generator_distinct()

        ####intra Attention#####
        for i in range(self.patch_nums):
            t = x[:, i * self.patch_size:(i + 1) * self.patch_size, :, :]

            intra_emb = self.embeddings_generator[i](self.intra_embeddings[i]).expand(batch_size, -1, -1,  -1)
            t = torch.cat([intra_emb, t], dim=1)
            out, attention = self.intra_patch_attention(intra_emb, t, t, weights_distinct, biases_distinct, weights_shared,
                                               biases_shared)

            if intra_out_concat == None:
                intra_out_concat = out

            else:
                intra_out_concat = torch.cat([intra_out_concat, out], dim=1)

        intra_out_concat = intra_out_concat.permute(0,3,2,1)
        intra_out_concat = self.intra_Linear(intra_out_concat)
        intra_out_concat = intra_out_concat.permute(0,3,2,1)

        ####inter Attention######
        print("Iter Attn Shapes")
        print(f"patch size: {self.patch_size}")
        print(f"x shape: {x.shape}")
        x = x.unfold(dimension=1, size=self.patch_size, step=self.stride)  # [b x patch_num x nvar x dim x patch_len]
        print(f"x shape: {x.shape}")
        x = x.permute(0, 2, 1, 3, 4)  # [b x nvar x patch_num x dim x patch_len ]
        print(f"x shape: {x.shape}")
        b, nvar, patch_num, dim, patch_len = x.shape

        x = torch.reshape(x, (
        x.shape[0] * x.shape[1], x.shape[2], x.shape[3] * x.shape[-1]))  # [b*nvar, patch_num, dim*patch_len]
        print(f"x shape: {x.shape}")
        x = self.emb_linear(x)
        print(f"x shape: {x.shape}")
        x = self.dropout(x + self.W_pos)

        inter_out, attention = self.inter_patch_attention(Q=x, K=x, V=x)  # [b*nvar, patch_num, dim]
        print(f"x shape: {x.shape}")
        inter_out = torch.reshape(inter_out, (b, nvar, inter_out.shape[-2], inter_out.shape[-1]))
        inter_out = torch.reshape(inter_out, (b, nvar, inter_out.shape[-2], self.patch_size, self.d_model))
        inter_out = torch.reshape(inter_out, (b, self.patch_size*self.patch_nums, nvar, self.d_model)) #[b, temporal, nvar, dim]

        print(f"new_x: {new_x.shape} | intra_out_concat: {intra_out_concat.shape} | inter_out: {inter_out.shape}")
        out = new_x + intra_out_concat + inter_out
        if self.batch_norm:
            out = self.norm_attn(out.reshape(b*nvar, self.patch_size*self.patch_nums, self.d_model))
        ##FFN
        out = self.dropout(out)
        out = self.ff(out) + out
        if self.batch_norm:
            out = self.norm_ffn(out).reshape(b, self.patch_size*self.patch_nums, nvar, self.d_model)
        return out, attention



class CustomLinear(nn.Module):
    def __init__(self, factorized):
        super(CustomLinear, self).__init__()
        self.factorized = factorized

    def forward(self, input, weights, biases):
        if self.factorized:
            return torch.matmul(input.unsqueeze(3), weights).squeeze(3) + biases
        else:
            return torch.matmul(input, weights) + biases


class Intra_Patch_Attention(nn.Module):
    def __init__(self, d_model, factorized):
        super(Intra_Patch_Attention, self).__init__()
        self.head = 2

        if d_model % self.head != 0:
            raise Exception('Hidden size is not divisible by the number of attention heads')

        self.head_size = int(d_model // self.head)
        self.custom_linear = CustomLinear(factorized)

    def forward(self, query, key, value, weights_distinct, biases_distinct, weights_shared, biases_shared):
        batch_size = query.shape[0]

        key = self.custom_linear(key, weights_distinct[0], biases_distinct[0])
        value = self.custom_linear(value, weights_distinct[1], biases_distinct[1])
        query = torch.cat(torch.split(query, self.head_size, dim=-1), dim=0)
        key = torch.cat(torch.split(key, self.head_size, dim=-1), dim=0)
        value = torch.cat(torch.split(value, self.head_size, dim=-1), dim=0)

        query = query.permute((0, 2, 1, 3))
        key = key.permute((0, 2, 3, 1))
        value = value.permute((0, 2, 1, 3))



        attention = torch.matmul(query, key)
        attention /= (self.head_size ** 0.5)

        attention = torch.softmax(attention, dim=-1)

        x = torch.matmul(attention, value)
        x = x.permute((0, 2, 1, 3))
        x = torch.cat(torch.split(x, batch_size, dim=0), dim=-1)

        if x.shape[0] == 0:
            x = x.repeat(1, 1, 1, int(weights_shared[0].shape[-1] / x.shape[-1]))

        x = self.custom_linear(x, weights_shared[0], biases_shared[0])
        x = torch.relu(x)
        x = self.custom_linear(x, weights_shared[1], biases_shared[1])
        return x, attention


class Inter_Patch_Attention(nn.Module):
    def __init__(self, d_model, out_dim, n_heads, d_k=None, d_v=None, res_attention=False, attn_dropout=0.,
                 proj_dropout=0., qkv_bias=True, lsa=False):
        super().__init__()
        d_k = d_model // n_heads if d_k is None else d_k
        d_v = d_model // n_heads if d_v is None else d_v

        self.n_heads, self.d_k, self.d_v = n_heads, d_k, d_v

        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=qkv_bias)

        # Scaled Dot-Product Attention (multiple heads)
        self.res_attention = res_attention
        self.sdp_attn = ScaledDotProductAttention(d_model, n_heads, attn_dropout=attn_dropout,
                                                  res_attention=self.res_attention, lsa=lsa)

        # Poject output
        self.to_out = nn.Sequential(nn.Linear(n_heads * d_v, out_dim), nn.Dropout(proj_dropout))


    def forward(self, Q, K=None, V=None, prev=None, key_padding_mask=None, attn_mask=None):

        bs = Q.size(0)
        if K is None: K = Q
        if V is None: V = Q

        # Linear (+ split in multiple heads)
        q_s = self.W_Q(Q).view(bs, Q.shape[1], self.n_heads, self.d_k).transpose(1,
                                                                                 2)  # q_s    : [bs x n_heads x q_len x d_k]  此处的q_len为patch_num
        k_s = self.W_K(K).view(bs, K.shape[1], self.n_heads, self.d_k).permute(0, 2, 3,
                                                                               1)  # k_s    : [bs x n_heads x d_k x q_len] - transpose(1,2) + transpose(2,3)
        v_s = self.W_V(V).view(bs, V.shape[1], self.n_heads, self.d_v).transpose(1,
                                                                                 2)  # v_s    : [bs x n_heads x q_len x d_v]

        # Apply Scaled Dot-Product Attention (multiple heads)
        if self.res_attention:
            output, attn_weights, attn_scores = self.sdp_attn(q_s, k_s, v_s, prev=prev,
                                                              key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        else:
            output, attn_weights = self.sdp_attn(q_s, k_s, v_s, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        output = output.transpose(1, 2).contiguous().view(bs, Q.shape[1],
                                                          self.n_heads * self.d_v)  # output: [bs x q_len x n_heads * d_v]
        output = self.to_out(output)

        return output, attn_weights


class ScaledDotProductAttention(nn.Module):
    r"""Scaled Dot-Product Attention module (Attention is all you need by Vaswani et al., 2017) with optional residual attention from previous layer
    (Realformer: Transformer likes residual attention by He et al, 2020) and locality self sttention (Vision Transformer for Small-Size Datasets
    by Lee et al, 2021)"""

    def __init__(self, d_model, n_heads, attn_dropout=0., res_attention=False, lsa=False):
        super().__init__()
        self.attn_dropout = nn.Dropout(attn_dropout)
        self.res_attention = res_attention
        head_dim = d_model // n_heads
        self.scale = nn.Parameter(torch.tensor(head_dim ** -0.5), requires_grad=lsa)
        self.lsa = lsa

    def forward(self, q, k, v, prev=None, key_padding_mask=None, attn_mask=None):

        # Scaled MatMul (q, k) - similarity scores for all pairs of positions in an input sequence
        attn_scores = torch.matmul(q, k) * self.scale  # attn_scores : [bs x n_heads x max_q_len x q_len]

        # Add pre-softmax attention scores from the previous layer (optional)
        if prev is not None: attn_scores = attn_scores + prev

        # Attention mask (optional)
        if attn_mask is not None:  # attn_mask with shape [q_len x seq_len] - only used when q_len == seq_len
            if attn_mask.dtype == torch.bool:
                attn_scores.masked_fill_(attn_mask, -np.inf)
            else:
                attn_scores += attn_mask

        # Key padding mask (optional)
        if key_padding_mask is not None:  # mask with shape [bs x q_len] (only when max_w_len == q_len)
            attn_scores.masked_fill_(key_padding_mask.unsqueeze(1).unsqueeze(2), -np.inf)

        # normalize the attention weights
        attn_weights = F.softmax(attn_scores, dim=-1)  # attn_weights   : [bs x n_heads x max_q_len x q_len]
        attn_weights = self.attn_dropout(attn_weights)

        # compute the new values given the attention weights
        output = torch.matmul(attn_weights, v)  # output: [bs x n_heads x max_q_len x d_v]

        return output, attn_weights


class WeightGenerator(nn.Module):
    def __init__(self, in_dim, out_dim, mem_dim, num_nodes, factorized, number_of_weights=4):
        super(WeightGenerator, self).__init__()
        #print('FACTORIZED {}'.format(factorized))
        self.number_of_weights = number_of_weights
        self.mem_dim = mem_dim
        self.num_nodes = num_nodes
        self.factorized = factorized
        self.out_dim = out_dim
        if self.factorized:
            self.memory = nn.Parameter(torch.randn(num_nodes, mem_dim), requires_grad=True).to('cpu')
            # self.memory = nn.Parameter(torch.randn(num_nodes, mem_dim), requires_grad=True).to('cuda:0')
            self.generator = self.generator = nn.Sequential(*[
                nn.Linear(mem_dim, 64),
                nn.Tanh(),
                nn.Linear(64, 64),
                nn.Tanh(),
                nn.Linear(64, 100)
            ])

            self.mem_dim = 10
            self.P = nn.ParameterList(
                [nn.Parameter(torch.Tensor(in_dim, self.mem_dim), requires_grad=True) for _ in
                 range(number_of_weights)])
            self.Q = nn.ParameterList(
                [nn.Parameter(torch.Tensor(self.mem_dim, out_dim), requires_grad=True) for _ in
                 range(number_of_weights)])
            self.B = nn.ParameterList(
                [nn.Parameter(torch.Tensor(self.mem_dim ** 2, out_dim), requires_grad=True) for _ in
                 range(number_of_weights)])
        else:
            self.P = nn.ParameterList(
                [nn.Parameter(torch.Tensor(in_dim, out_dim), requires_grad=True) for _ in range(number_of_weights)])
            self.B = nn.ParameterList(
                [nn.Parameter(torch.Tensor(1, out_dim), requires_grad=True) for _ in range(number_of_weights)])
        self.reset_parameters()

    def reset_parameters(self):
        list_params = [self.P, self.Q, self.B] if self.factorized else [self.P]
        for weight_list in list_params:
            for weight in weight_list:
                init.kaiming_uniform_(weight, a=math.sqrt(5))

        if not self.factorized:
            for i in range(self.number_of_weights):
                fan_in, _ = init._calculate_fan_in_and_fan_out(self.P[i])
                bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
                init.uniform_(self.B[i], -bound, bound)

    def forward(self):
        if self.factorized:
            memory = self.generator(self.memory.unsqueeze(1))
            bias = [torch.matmul(memory, self.B[i]).squeeze(1) for i in range(self.number_of_weights)]
            memory = memory.view(self.num_nodes, self.mem_dim, self.mem_dim)
            weights = [torch.matmul(torch.matmul(self.P[i], memory), self.Q[i]) for i in range(self.number_of_weights)]
            return weights, bias
        else:
            return self.P, self.B



class Transpose(nn.Module):
    def __init__(self, *dims, contiguous=False):
        super().__init__()
        self.dims, self.contiguous = dims, contiguous
    def forward(self, x):
        if self.contiguous: return x.transpose(*self.dims).contiguous()
        else: return x.transpose(*self.dims)

In [99]:
import torch
import torch.nn as nn

class RevIN(nn.Module):
    def __init__(self, num_features: int, eps=1e-5, affine=True, subtract_last=False):
        """
        :param num_features: the number of features or channels
        :param eps: a value added for numerical stability
        :param affine: if True, RevIN has learnable affine parameters
        """
        super(RevIN, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        self.subtract_last = subtract_last
        if self.affine:
            self._init_params()

    def forward(self, x, mode:str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        else: raise NotImplementedError
        return x

    def _init_params(self):
        # initialize RevIN params: (C,)
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def _get_statistics(self, x):
        dim2reduce = tuple(range(1, x.ndim-1))
        if self.subtract_last:
            self.last = x[:,-1,:].unsqueeze(1)
        else:
            self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        if self.subtract_last:
            x = x - self.last
        else:
            x = x - self.mean
        x = x / self.stdev
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.affine:
            x = x - self.affine_bias
            x = x / (self.affine_weight + self.eps*self.eps)
        x = x * self.stdev
        if self.subtract_last:
            x = x + self.last
        else:
            x = x + self.mean
        return x

In [100]:
import torch
import torch.nn as nn
from torch.distributions.normal import Normal


class AMS(nn.Module):
    def __init__(self, input_size, output_size, num_experts, device, num_nodes=1, d_model=32, d_ff=64, dynamic=False,
                 patch_size=[8, 6, 4, 2], noisy_gating=True, k=4, layer_number=1, residual_connection=1, batch_norm=False):
        super(AMS, self).__init__()
        self.num_experts = num_experts
        self.output_size = output_size
        self.input_size = input_size
        self.k = k

        self.start_linear = nn.Linear(in_features=num_nodes, out_features=1)
        self.seasonality_model = FourierLayer(pred_len=0, k=3)
        self.trend_model = series_decomp_multi(kernel_size=[4, 8, 12])

        self.experts = nn.ModuleList()
        self.MLPs = nn.ModuleList()
        for patch in patch_size:
            patch_nums = int(input_size / patch)
            self.experts.append(Transformer_Layer(device=device, d_model=d_model, d_ff=d_ff,
                                      dynamic=dynamic, num_nodes=num_nodes, patch_nums=patch_nums,
                                      patch_size=patch, factorized=True, layer_number=layer_number, batch_norm=batch_norm))

        # self.w_gate = nn.Parameter(torch.zeros(input_size, num_experts), requires_grad=True)
        # self.w_noise = nn.Parameter(torch.zeros(input_size, num_experts), requires_grad=True)
        self.w_noise = nn.Linear(input_size, num_experts)
        self.w_gate = nn.Linear(input_size, num_experts)

        self.residual_connection = residual_connection
        self.end_MLP = MLP(input_size=input_size, output_size=output_size)

        self.noisy_gating = noisy_gating
        self.softplus = nn.Softplus()
        self.softmax = nn.Softmax(1)
        self.register_buffer("mean", torch.tensor([0.0]))
        self.register_buffer("std", torch.tensor([1.0]))
        assert (self.k <= self.num_experts)

    def cv_squared(self, x):
        eps = 1e-10
        if x.shape[0] == 1:
            return torch.tensor([0], device=x.device, dtype=x.dtype)
        return x.float().var() / (x.float().mean() ** 2 + eps)

    def _gates_to_load(self, gates):
        return (gates > 0).sum(0)

    def _prob_in_top_k(self, clean_values, noisy_values, noise_stddev, noisy_top_values):
        batch = clean_values.size(0)
        m = noisy_top_values.size(1)
        top_values_flat = noisy_top_values.flatten()

        threshold_positions_if_in = torch.arange(batch, device=clean_values.device) * m + self.k
        threshold_if_in = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_in), 1)
        is_in = torch.gt(noisy_values, threshold_if_in)
        threshold_positions_if_out = threshold_positions_if_in - 1
        threshold_if_out = torch.unsqueeze(torch.gather(top_values_flat, 0, threshold_positions_if_out), 1)
        normal = Normal(self.mean, self.std)
        prob_if_in = normal.cdf((clean_values - threshold_if_in) / noise_stddev)
        prob_if_out = normal.cdf((clean_values - threshold_if_out) / noise_stddev)
        prob = torch.where(is_in, prob_if_in, prob_if_out)
        return prob

    def seasonality_and_trend_decompose(self, x):
        x = x[:, :, :, 0]
        _, trend = self.trend_model(x)
        seasonality, _ = self.seasonality_model(x)
        return x + seasonality + trend

    def noisy_top_k_gating(self, x, train, noise_epsilon=1e-2):
        #print(f"x shape: {x.shape}")
        x = self.start_linear(x).squeeze(-1)

        # clean_logits = x @ self.w_gate
        clean_logits = self.w_gate(x)
        if self.noisy_gating and train:
            # raw_noise_stddev = x @ self.w_noise
            raw_noise_stddev = self.w_noise(x)
            noise_stddev = ((self.softplus(raw_noise_stddev) + noise_epsilon))
            noisy_logits = clean_logits + (torch.randn_like(clean_logits) * noise_stddev)
            logits = noisy_logits
        else:
            logits = clean_logits
        # calculate topk + 1 that will be needed for the noisy gates
        top_logits, top_indices = logits.topk(min(self.k + 1, self.num_experts), dim=1)

        top_k_logits = top_logits[:, :self.k]
        top_k_indices = top_indices[:, :self.k]
        top_k_gates = self.softmax(top_k_logits)

        zeros = torch.zeros_like(logits, requires_grad=True)
        gates = zeros.scatter(1, top_k_indices, top_k_gates)

        if self.noisy_gating and self.k < self.num_experts and train:
            load = (self._prob_in_top_k(clean_logits, noisy_logits, noise_stddev, top_logits)).sum(0)
        else:
            load = self._gates_to_load(gates)
        return gates, load

    def forward(self, x, loss_coef=1e-2):
        new_x = self.seasonality_and_trend_decompose(x)

        #multi-scale router
        gates, load = self.noisy_top_k_gating(new_x, self.training)
        # calculate balance loss
        importance = gates.sum(0)
        balance_loss = self.cv_squared(importance) + self.cv_squared(load)
        balance_loss *= loss_coef
        dispatcher = SparseDispatcher(self.num_experts, gates)
        expert_inputs = dispatcher.dispatch(x)
        expert_outputs = [self.experts[i](expert_inputs[i])[0] for i in range(self.num_experts)]
        output = dispatcher.combine(expert_outputs)
        if self.residual_connection:
            output = output + x
        return output, balance_loss

In [101]:
import math
import torch
import torch
import torch.nn as nn
from torch.distributions.normal import Normal
import numpy as np
from functools import reduce
from operator import mul


class Model(nn.Module):
    def __init__(self, configs):
        super(Model, self).__init__()
        self.layer_nums = configs.layer_nums  # 设置pathway的层数
        self.num_nodes = configs.num_nodes
        self.pre_len = configs.pred_len
        self.seq_len = configs.seq_len
        self.k = configs.k
        self.num_experts_list = configs.num_experts_list
        self.patch_size_list = configs.patch_size_list
        self.d_model = configs.d_model
        self.d_ff = configs.d_ff
        self.residual_connection = configs.residual_connection
        self.revin = configs.revin
        if self.revin:
            self.revin_layer = RevIN(num_features=configs.num_nodes, affine=False, subtract_last=False)

        self.start_fc = nn.Linear(in_features=1, out_features=self.d_model)
        self.AMS_lists = nn.ModuleList()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.batch_norm = configs.batch_norm

        for num in range(self.layer_nums):
            self.AMS_lists.append(
                AMS(self.seq_len, self.seq_len, self.num_experts_list[num], self.device, k=self.k,
                    num_nodes=self.num_nodes, patch_size=[self.patch_size_list[num]], noisy_gating=True,
                    d_model=self.d_model, d_ff=self.d_ff, layer_number=num + 1, residual_connection=self.residual_connection, batch_norm=self.batch_norm))
        self.projections = nn.Sequential(
            nn.Linear(self.seq_len * self.d_model, self.pre_len)
        )

    def forward(self, x):

        balance_loss = 0
        # norm
        if self.revin:
            x = self.revin_layer(x, 'norm')
        out = self.start_fc(x.unsqueeze(-1))


        batch_size = x.shape[0]

        for layer in self.AMS_lists:
            out, aux_loss = layer(out)
            balance_loss += aux_loss

        out = out.permute(0,2,1,3).reshape(batch_size, self.num_nodes, -1)
        out = self.projections(out).transpose(2, 1)

        # denorm
        if self.revin:
            out = self.revin_layer(out, 'denorm')

        return out, balance_loss

In [102]:
import torch

def classification_evaluate_direction(output, batch_y):
    """
    Evaluate the accuracy of direction predictions and compare it to a baseline model.

    Parameters:
    output (torch.Tensor): Tensor of predicted values with shape (batch_size, 96, 1).
    batch_y (torch.Tensor): Tensor of true values with shape (batch_size, 96, 1).

    Returns:
    correct_predictions (int): Number of times the predicted direction was correct.
    accuracy (float): Fraction of correct predictions out of total comparisons.
    baseline_correct (int): Number of times the baseline prediction was correct.
    baseline_accuracy (float): Fraction of correct predictions by the baseline out of total comparisons.
    """
    # Compute the direction of change for each tensor
    def compute_direction(tensor):
        # Compute the difference between consecutive elements
        diff = tensor[:, 1:, :] - tensor[:, :-1, :]
        # Convert differences to binary direction indicators: 1 for increase, -1 for decrease
        direction = torch.sign(diff)
        return direction

    # Get direction indicators for both tensors
    #print(batch_y[:,0,:].shape)
    output = torch.cat((batch_y[:, 0, :].unsqueeze(2), output), dim=1)
    #print(output)
    output_direction = compute_direction(output)
    batch_y_direction = compute_direction(batch_y)
    #print(f"output_direction: {output_direction.shape} batch_y_direction: {batch_y_direction.shape}")
    # Compare the directions
    correct_predictions = (output_direction == batch_y_direction).sum().item()
    #print(f"correct_predictions: {correct_predictions}")
    total_comparisons = output_direction.numel()
    #print(f"total_comparisons: {total_comparisons}")
    accuracy = correct_predictions / total_comparisons

    # Compute baseline predictions (always predict increase)
    baseline_direction = torch.ones_like(output_direction)  # Baseline always predicts increase
    baseline_correct = (baseline_direction == batch_y_direction).sum().item()
    baseline_accuracy = baseline_correct / total_comparisons

    return correct_predictions, baseline_correct, total_comparisons

In [103]:
def compute_direction(tensor):
  batch_size = tensor.size(0)
  comparison = tensor[:, 1, 0] > tensor[:, 0, 0]
  # Convert the comparison result to a one-hot vector
  one_hot = torch.zeros(batch_size, 2)
  one_hot[comparison, 0] = 1  # [1, 0] when the last value is greater
  one_hot[~comparison, 1] = 1  # [0, 1] when the first value is greater
  return one_hot

tensor = torch.randn(5, 2, 1)
print(tensor)
compute_direction(tensor)

tensor([[[-1.4507],
         [-0.2593]],

        [[ 1.1515],
         [-0.1691]],

        [[-0.8981],
         [-0.8005]],

        [[-0.5312],
         [ 1.0946]],

        [[ 0.4032],
         [ 0.8267]]])


tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

# Prediction Test

In [104]:
from dataclasses import dataclass, field
@dataclass
class Args():
    freq: str = 'd'
    task_name: str = 'forecasting'
    num_class: int = 2
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    d_model: int = 16
    d_ff: int = 128
    patch_len: int = 16
    moving_avg: int = 25
    factor: int = 3
    distil: bool = True
    output_attention: bool = False
    patience: int = 400
    stride: int = 1
    learning_rate: float = 0.0005
    batch_size: int = 32
    embed: str = 'timeF'
    dropout: float = 0.0
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    train_epochs: int = 100
    use_statistic: bool = False
    mask_rate: float = 0.25
    anomaly_ratio: float = 0.25
    num_kernels: int = 6
    moving_avg: int = 25
    activation: str = 'gelu'
    fc_dropout: float = 0.3
    head_dropout: float = 0.3
    momentum: float = 0.1
    dp_rank: int = 8
    merge_size: int = 2
    alpha: float = 0.5
    beta: float = 0.5
    layer_nums: int = 3
    num_nodes: int = 32
    k: int = 2
    residual_connection: int = 2
    num_experts_list: list = field(default_factory=lambda: [4, 4, 4])
    patch_size_list: list = field(default_factory=lambda: [16,12,8,32,12,8,6,4,8,6,4,2])
    revin: int = 1
    batch_norm: int = 0
    drop: float = 0.1

    ## Data
    batch_size: int = 3
    data_start_year: int = 1990
    data_end_year: int = 2023
    scale: bool = True
    one_hot_datetime: bool = False
    datetime_features: int = 55

    ## Training
    run_name: str = "test"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "^SPX" #"goog amzn wmt xom brk-a lly ge lin pld aapl nee"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "pathformer_rolling"

In [105]:
configs = Args()
pathformer = Model(configs)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_optim = torch.optim.Adam(pathformer.parameters(), lr=configs.learning_rate)
loss_fn = nn.BCEWithLogitsLoss()
dataset = StockDataset(tickers='^SPX',timeenc=1, freq='d', size=[36, 18, 1], features='MS', scale=configs.scale, batch_size=configs.batch_size)

Loading following tickers: ['^SPX']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]
Normalizing Data: True
DateTime is one-hot: False


In [106]:
device = "cuda" if torch.cuda.is_available() else "cpu"
range_limit = configs.rolling_window+configs.validation_years+configs.test_years
loop_range = tqdm.tqdm(range(len(dataset)-range_limit))
pathformer = pathformer.to(device)
for iteration in loop_range:
  for window_iteration in range(configs.window_epoch):
    epoch_loss = []
    total_hits = 0
    total_data = 0
    total_ol = 0
    starting_idx = iteration // 10
    for i in range((configs.rolling_window+configs.validation_years+configs.test_years)):
        validation, test = False, False
        if i == configs.rolling_window:
          validation = True
        elif i == (configs.rolling_window+1):
          test = True
        else:
          batches, year  = dataset[i+iteration]
          batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
          for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
            model_optim.zero_grad()
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)
            true_batch_y = batch_y.float().to(device)
            batch_x_mark = batch_x_mark.float().to(device)
            batch_y_mark = batch_y_mark.float().to(device)
            dec_inp = torch.zeros_like(true_batch_y[:, -configs.pred_len:, :]).float()
            dec_inp = torch.cat([true_batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(device)
            #print(batch_x.shape)

            #print(f"batch_x: {batch_x}")
            outputs = pathformer(batch_x)
            print(f"outputs shape")


            f_dim = -1 if configs.features == 'MS' else 0
            batch_y_direction = batch_y[:, -(configs.pred_len+1):, f_dim:].to(device)
            print(f"batch_y_direction: {batch_y_direction}")
            batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(device)
            truth_direction = compute_direction(batch_y_direction).to(device)
            print(f"outputs: {outputs.shape} | truth_direction: {truth_direction.shape}")
            print(f"truth_direction: {truth_direction}")
            loss = loss_fn(outputs, truth_direction)
            epoch_loss.append(loss.item())
            loss.backward()
            model_optim.step()

            p = outputs.detach().cpu().numpy().argmax(axis=1)
            l = truth_direction.detach().cpu().numpy().argmax(axis=1)
            only_long = np.zeros_like(l)
            correct_preds = np.sum(p == l)
            only_longs = np.sum(only_long == l)

            #print(f"outputs shape: {outputs.shape} | true_batch_y[:,-97:,:]: {true_batch_y[:,-97:,-1:].shape}")
            total_hits += correct_preds
            total_data += outputs.size(0)
            total_ol += only_longs

            raise ValueError("Stopped on Purpose")
        print(f"Year: {year} total_comparisons: {total_data} training_hit_ratio: {total_hits/total_data} | ol_hit_ratio: {total_ol/total_data}")
        break
    break
  break

  0%|          | 0/21 [00:00<?, ?it/s]

x shape: torch.Size([0, 36, 32, 16])
Iter Attn Shapes
pathc size: 16
x shape: torch.Size([0, 36, 32, 16])
x shape: torch.Size([0, 2, 32, 16, 16])
x shape: torch.Size([0, 32, 2, 16, 16])
x shape: torch.Size([0, 2, 256])
x shape: torch.Size([0, 2, 256])
x shape: torch.Size([0, 2, 256])
new_x: torch.Size([0, 36, 32, 16]) | intra_out_concat: torch.Size([0, 32, 32, 16]) | inter_out: torch.Size([0, 32, 32, 16])


RuntimeError: The size of tensor a (36) must match the size of tensor b (32) at non-singleton dimension 1

# Trainer

In [ ]:
class Trainer():
  def __init__(self, configs):
    self.configs = configs

    ticker_str = configs.tickers.replace(" ", "_")
    self.run_name = f"[{ticker_str}]_valYrs:{self.configs.validation_years}_testYrs{self.configs.test_years}_scale:{self.configs.scale}_reset:{self.configs.reset_model}_{configs.run_name}"
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.set_seed(configs.seed)
    self.dataset = self.make_data()
    self.model = Model(configs)
    self.model.to(self.device)
    self.loss_fn = nn.BCEWithLogitsLoss()
    self.model_optim = torch.optim.Adam(self.model.parameters(), lr=configs.learning_rate)
    self.writer = SummaryWriter(f"/content/drive/MyDrive/code/fintransformer/runs/{self.run_name}")
    path = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}/{self.run_name}"
    if not os.path.exists(path):
      os.mkdir(path)
      print(f"Save File Directory Made at {path}\n")
    else:
      print(f"Directory Already Exists at {path}")

  def set_seed(self, seed):
      torch.manual_seed(seed)
      torch.cuda.manual_seed(seed)
      #torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
      np.random.seed(seed)
      random.seed(seed)
      torch.backends.cudnn.deterministic = True
      torch.backends.cudnn.benchmark = False

  def make_data(self):
    def collate_fn(batch):
      seq_x, seq_y, seq_x_mark, seq_y_mark, year = batch[0]
      return seq_x, seq_y, seq_x_mark, seq_y_mark, year
    dataset = StockDataset(tickers=self.configs.tickers, timeenc=1, freq='d', size=[self.configs.seq_len, self.configs.label_len, self.configs.pred_len],
                           features=self.configs.features, batch_size=self.configs.batch_size, scale=self.configs.scale,
                           data_start_year=self.configs.data_start_year, data_end_year=self.configs.data_end_year)
    """prices, labels, year = next(iter(val_loader))
    feature_num = prices.shape[2]
    label_num = labels.shape[1]
    print(f"Data shape: {prices.shape} | Num Features: {feature_num}")"""
    return dataset

  def run(self):
    ### Training ###
    print(f"Training model: {self.configs.run_name}")
    print()
    range_limit = self.configs.rolling_window + self.configs.validation_years + self.configs.test_years
    loop_range = tqdm.tqdm(range(len(self.dataset)-range_limit))
    recent_save_path = ""
    val_acc_arr = []
    test_acc_arr = []
    for iteration in loop_range:
      highest_acc = 0.0
      highest_test_acc = 0.0
      if self.configs.reset_model and iteration != 0:
        self.model = Model(self.configs)
        self.model.to(self.device)
        self.model_optim = torch.optim.Adam(self.model.parameters(), lr=self.configs.learning_rate)
        print(f"Model Has Been Reset to Random Parameters")
      else:
        self.load_model(recent_save_path) if recent_save_path != "" else None
      for window_iteration in range(self.configs.window_epoch):
        train_loss, average_val_loss, average_val_accuracy, average_test_accuracy, validation_year, test_year = self.train(iteration, window_iteration, highest_acc)
        if average_val_accuracy > highest_acc:
            highest_acc = average_val_accuracy
            highest_test_acc = average_test_accuracy
            recent_save_path = self.save_model(average_val_accuracy, average_test_accuracy, window_iteration, validation_year, test_year)
      #self.writer.add_scalar("Train/Test Loss", train_loss, iteration)
      val_acc_arr.append([highest_acc, validation_year])
      test_acc_arr.append([highest_test_acc, test_year])
      print(f"***********************************************************")
      print(f"Highest Validation Accuracy in {validation_year[0]}~{validation_year[-1]}: {highest_acc*100:.2f}% | Highest Test Accuracy in {test_year}: {average_test_accuracy*100:.2f}%")
      print(f"***********************************************************")
      print("=============================New Training Set====================================")
    self.writer.flush()
    self.writer.close()
    #print(f"Highest Accuracy in Validation Set: {highest_acc*100:.2f}")
    print()
    for val, test in zip(val_acc_arr, test_acc_arr):
      print(f"Validation Accuracy in {val[1][0]}~{val[1][-1]} : {val[0]*100:.2f}% | Test Accuracy in {test[1]}: {test[0]*100:.2f}%")

    average_first_elements_val = sum(item[0] for item in val_acc_arr) / len(val_acc_arr)
    average_first_elements_test = sum(item[0] for item in test_acc_arr) / len(test_acc_arr)
    print(f"Average Validation Accuracy: {average_first_elements_val*100:.2f}% | Average Test Accuracy: {average_first_elements_test*100:.2f}%")
    return

  def train(self, iteration, window_iteration, highest_acc):
    epoch_loss = []
    val_epoch_loss = []
    training_start_year, training_end_year = 0, 0
    training_total_hits = 0
    training_total_data = 0
    training_total_ol = 0
    validation_year, test_year = [], 0
    validation_total_hits = 0
    validation_total_data = 0
    validation_total_ol = 0
    test_total_hits = 0
    test_total_data = 0
    test_total_ol = 0
    for i in range((self.configs.rolling_window+self.configs.validation_years+self.configs.test_years)):
      validation, test = False, False
      if i >= self.configs.rolling_window and i <= (self.configs.rolling_window+self.configs.validation_years-1):
        batches, year = self.dataset[i+iteration]
        #print(f"validation year: {year}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        accuracy, only_long, val_loss_epoch, total_hits, total_data, total_ol = self.eval_once(batches_x, batches_y, batches_x_mark, batches_y_mark, year, highest_acc=0, train_acc=0)
        validation_total_hits += total_hits
        validation_total_data += total_data
        validation_total_ol += total_ol
        val_epoch_loss.append(val_loss_epoch)
        validation_year.append(year)
        validation = True
        print(f"Year: {year} | Validation Accuracy: {(total_hits/total_data)*100:.2f}% | Only Long Accuracy: {(total_ol/total_data)*100:.2f}% | Highest Validation Accuracy: {highest_acc*100:.2f}%")
      elif i >= (self.configs.rolling_window+self.configs.validation_years):
        batches, year = self.dataset[i+iteration]
        #print(f"test year: {year}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        accuracy, only_long,  total_hits, total_data, total_ol = self.test(batches_x, batches_y, batches_x_mark, batches_y_mark, year)
        test_total_hits += total_hits
        test_total_data += total_data
        test_total_ol += total_ol
        test_year = year
        test = True
      else:
        self.model.train()
        batches, year = self.dataset[i+iteration]
        training_start_year = year if i == 0 else training_start_year
        training_end_year = year if i == self.configs.rolling_window-1 else training_end_year
        #print(f"train year: {year}")
        batches_x, batches_y, batches_x_mark, batches_y_mark = batches[0], batches[1], batches[2], batches[3]
        for j, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
          self.model_optim.zero_grad()

          batch_x = batch_x.float().to(self.device)
          batch_y = batch_y.float().to(self.device)
          true_batch_y = batch_y.float().to(self.device)
          batch_x_mark = batch_x_mark.float().to(self.device)
          batch_y_mark = batch_y_mark.float().to(self.device)
          #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
          dec_inp = torch.zeros_like(true_batch_y[:, -configs.pred_len:, :]).float()
          dec_inp = torch.cat([true_batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(self.device)
          #print(batch_x.shape)

          outputs = self.model(batch_x, None, dec_inp, None)

          f_dim = -1 if configs.features == 'MS' else 0
          batch_y_direction = batch_y[:, -(configs.pred_len+1):, f_dim:].to(self.device)
          batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(self.device)
          truth_direction = compute_direction(batch_y_direction).to(self.device)
          #print(f"outputs: {outputs.shape} | truth_direction: {truth_direction.shape}")
          loss = self.loss_fn(outputs, truth_direction)
          epoch_loss.append(loss.item())
          loss.backward()
          self.model_optim.step()

          p = outputs.detach().cpu().numpy().argmax(axis=1)
          l = truth_direction.detach().cpu().numpy().argmax(axis=1)
          only_long = np.zeros_like(l)
          correct_preds = np.sum(p == l)
          only_longs = np.sum(only_long == l)
          #print(f"outputs: {outputs.shape} | batch_y: {batch_y.shape}")
          #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)

          training_total_hits += correct_preds
          training_total_data += outputs.size(0)
          training_total_ol += only_longs

    loss_epoch = np.mean(epoch_loss)
    average_val_accuracy = validation_total_hits / validation_total_data
    average_val_only_long = validation_total_ol / validation_total_data
    average_val_loss = np.mean(val_epoch_loss)
    average_test_accuracy = test_total_hits / test_total_data
    average_test_only_long = test_total_ol / test_total_data
    train_accuracy = training_total_hits / training_total_data
    train_only_long = training_total_ol / training_total_data
    print(f"Training Years: {training_start_year}~{training_end_year} | Training Accuracy: {train_accuracy*100:.2f}% | Training OL Accuracy: {train_only_long*100:.2f}%")
    print(f"Year: {validation_year[0]}~{validation_year[-1]} Validation Accuracy: {average_val_accuracy*100:.2f}% | Only Long Accuracy: {average_val_only_long*100:.2f}% | Highest Validation Accuracy: {highest_acc*100:.2f}%")
    print(f"Year: {test_year} | Test Accuracy: {average_test_accuracy*100:.2f}% | Only Long Accuracy: {average_test_only_long*100:.2f}%")
    print(f"------------------------------------------------")
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Train Accuracy", train_accuracy, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Train Loss", loss_epoch, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Validation Loss", average_val_loss, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Validation Accuracy", average_val_accuracy, window_iteration)
    self.writer.add_scalar(f"Train:{training_start_year}~{training_end_year}/Test Accuracy", average_test_accuracy, window_iteration)
      #print(f"Year: {year} training_hit_ratio: {hit_count/batch_x.shape[0]} | ol_hit_ratio: {only_long/batch_x.shape[0]}")

    #print(f"train accuracy: {accuracy*100:.2f}% | only_long: {only_long*100:.2f}%")
    return loss_epoch, average_val_loss, average_val_accuracy, average_test_accuracy, validation_year, test_year

  def eval_once(self, batches_x, batches_y, batches_x_mark, batches_y_mark, year, highest_acc, train_acc, last=False):
    self.model.eval()
    epoch_loss = []
    total_hits = 0
    total_data = 0
    total_ol = 0
    with torch.no_grad():
      for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        true_batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)
        #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
        dec_inp = torch.zeros_like(true_batch_y[:, -configs.pred_len:, :]).float()
        dec_inp = torch.cat([true_batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(self.device)
        #print(batch_x.shape)

        outputs = self.model(batch_x, None, dec_inp, None)

        f_dim = -1 if configs.features == 'MS' else 0
        batch_y_direction = batch_y[:, -(configs.pred_len+1):, f_dim:].to(self.device)
        batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(self.device)
        truth_direction = compute_direction(batch_y_direction).to(self.device)


        loss = self.loss_fn(outputs, truth_direction)
        epoch_loss.append(loss.item())

        p = outputs.detach().cpu().numpy().argmax(axis=1)
        l = truth_direction.detach().cpu().numpy().argmax(axis=1)
        only_long = np.zeros_like(l)
        correct_preds = np.sum(p == l)
        only_longs = np.sum(only_long == l)

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        total_hits += correct_preds
        total_data += outputs.size(0)
        total_ol += only_longs
        #print(f"Year: {year} hit_ratio: {(hit_count/count)*100:.2f}% | ol_hit_ratio: {(only_long/count)*100:.2f}%")

    accuracy = total_hits / total_data
    long_strategy = total_ol / total_data
    loss_epoch = np.mean(epoch_loss)
    #print(f"validation accuracy: {accuracy*100:.2f}% | val highest_acc: {highest_acc*100:.2f}% | val only long strategy: {long_strategy*100:.2f}%")
    return accuracy, long_strategy, loss_epoch, total_hits, total_data, total_ol

  def test(self, batches_x, batches_y, batches_x_mark, batches_y_mark, year):
    self.model.eval()
    total_hits = 0
    total_data = 0
    total_ol = 0
    with torch.no_grad():
      for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(zip(batches_x, batches_y, batches_x_mark, batches_y_mark)):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float().to(self.device)
        true_batch_y = batch_y.float().to(self.device)
        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)
        #print(f"batch_x: {batch_x.shape} batch_y: {batch_y.shape} batch_x_mark: {batch_x_mark.shape} batch_y_mark: {batch_y_mark.shape}")
        dec_inp = torch.zeros_like(true_batch_y[:, -configs.pred_len:, :]).float()
        dec_inp = torch.cat([true_batch_y[:, :configs.label_len, :], dec_inp], dim=1).float().to(self.device)
        #print(batch_x.shape)

        outputs = self.model(batch_x, None, dec_inp, None)

        f_dim = -1 if configs.features == 'MS' else 0
        batch_y_direction = batch_y[:, -(configs.pred_len+1):, f_dim:].to(self.device)
        batch_y = batch_y[:, -configs.pred_len:, f_dim:].to(self.device)
        truth_direction = compute_direction(batch_y_direction).to(self.device)

        p = outputs.detach().cpu().numpy().argmax(axis=1)
        l = truth_direction.detach().cpu().numpy().argmax(axis=1)
        only_long = np.zeros_like(l)
        correct_preds = np.sum(p == l)
        only_longs = np.sum(only_long == l)

        #hit_count, only_long, count = calculate_hit(outputs, batch_y, configs.label_len-1)
        total_hits += correct_preds
        total_data += outputs.size(0)
        total_ol += only_longs
        #print(f"Year: {year} hit_ratio: {(hit_count/count)*100:.2f}% | ol_hit_ratio: {(only_long/count)*100:.2f}%")

    accuracy = total_hits / total_data
    long_strategy = total_ol / total_data
    #print(f"test accuracy: {accuracy*100:.2f}% | test only long strategy: {long_strategy*100:.2f}%")
    return accuracy, long_strategy, total_hits, total_data, total_ol

  def save_model(self, val_acc, test_acc, epoch, validation_year, test_year):
    PATH = f"/content/drive/MyDrive/code/fintransformer/models/{self.configs.save_folder}/{self.run_name}/reset:{self.configs.reset_model}_valYear:{validation_year}_testYear:{test_year}.pt"
    torch.save({
            'model_state_dict': self.model.state_dict(),
            'model_optimizer_state_dict': self.model_optim.state_dict(),
            'val_acc': val_acc,
            'validation_year': validation_year,
            'test_acc': test_acc,
            'test_year': test_year,
            'configs': self.configs}, PATH)
    print(f"Model Saved at {PATH}")
    return PATH

  def load_model(self, path):
    checkpoint = torch.load(path, map_location=torch.device(self.device))
    self.model.load_state_dict(checkpoint['model_state_dict'])
    self.model_optim.load_state_dict(checkpoint['model_optimizer_state_dict'])
    print(f"Model loaded from {path}")
    print(f"Model Validation Accuracy: {checkpoint['val_acc']*100:.2f}% | Test Accuracy: {checkpoint['test_acc']*100:.2f}%")
    try:
      self.configs = checkpoint['configs']
    except:
      print("No Configs Found in torch file")
    return

### Not One-hot encoding

In [ ]:
from dataclasses import dataclass
@dataclass
class Args():
    freq: str = 'd'
    task_name: str = 'classification'
    num_class: int = 2
    seq_len: int = 36
    label_len: int = 18
    pred_len: int = 1
    e_layers: int = 2
    d_layers: int = 1
    n_heads: int = 16
    top_k: int = 5
    factor: int = 1
    enc_in: int = 32
    dec_in: int = 32
    c_out: int = 1
    d_model: int = 128
    d_ff: int = 512
    patch_len: int = 16
    moving_avg: int = 25
    factor: int = 3
    distil: bool = True
    output_attention: bool = False
    patience: int = 400
    stride: int = 1
    learning_rate: float = 0.0005
    batch_size: int = 32
    embed: str = 'timeF'
    activation: str = 'gelu'
    dropout: float = 0.0
    loss: str = 'mse'
    data: str = 'custom'
    features: str = 'MS'
    train_epochs: int = 100
    use_statistic: bool = False
    mask_rate: float = 0.25
    anomaly_ratio: float = 0.25
    num_kernels: int = 6
    moving_avg: int = 25
    activation: str = 'gelu'
    fc_dropout: float = 0.3
    head_dropout: float = 0.3
    momentum: float = 0.1
    dp_rank: int = 8
    merge_size: int = 2
    alpha: float = 0.5
    beta: float = 0.5

    ## Data
    batch_size: int = 32
    data_start_year: int = 1990
    data_end_year: int = 2023
    scale: bool = True
    one_hot_datetime: bool = False
    datetime_features: int = 55

    ## Training
    run_name: str = "scale_card_classification_run4"
    seed: int = 2024
    validation_years: int = 2
    test_years: int = 1
    tickers: str = "^SPX" #"goog amzn wmt xom brk-a lly ge lin pld aapl nee"
    rolling_window: int = 10 # How many training years to be included in each training dataset
    window_epoch: int = 50 # How many epochs to train per dataset
    reset_model: bool = False
    save_folder: str = "card_rolling"

In [ ]:
configs = Args()
trainer = Trainer(configs)

Loading following tickers: ['^SPX']

Dataset Start Year: 1990 | End Year: 2023
years: [1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002
 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021 2022 2023]
Normalizing Data: True
DateTime is one-hot: False
Save File Directory Made at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:True_reset:False_scale_card_classification_run4



In [ ]:
trainer.run()

Training model: scale_card_classification_run4



  0%|          | 0/21 [00:00<?, ?it/s]

Year: 2000 | Validation Accuracy: 57.14% | Only Long Accuracy: 47.62% | Highest Validation Accuracy: 0.00%
Year: 2001 | Validation Accuracy: 52.82% | Only Long Accuracy: 47.98% | Highest Validation Accuracy: 0.00%
Training Years: 1990~1999 | Training Accuracy: 53.56% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 55.00% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 0.00%
Year: 2002 | Test Accuracy: 51.98% | Only Long Accuracy: 44.44%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:True_reset:False_scale_card_classification_run4/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Year: 2000 | Validation Accuracy: 51.19% | Only Long Accuracy: 47.62% | Highest Validation Accuracy: 55.00%
Year: 2001 | Validation Accuracy: 54.84% | Only Long Accuracy: 47.98% | Highest Validation Accuracy: 55.00%
Training Years: 1990~1999 | Training Accuracy: 57.08% |

  5%|▍         | 1/21 [03:13<1:04:20, 193.05s/it]

Training Years: 1990~1999 | Training Accuracy: 76.54% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 50.00% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 55.00%
Year: 2002 | Test Accuracy: 50.40% | Only Long Accuracy: 44.44%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2000~2001: 55.00% | Highest Test Accuracy in 2002: 50.40%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:True_reset:False_scale_card_classification_run4/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Model Validation Accuracy: 55.00% | Test Accuracy: 51.98%


...

Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:True_reset:False_scale_card_classification_run4/reset:False_valYear:[2015, 2016]_testYear:2017.pt
Model Validation Accuracy: 57.14% | Test Accuracy: 52.19%
Year: 2016 | Validation Accuracy: 55.56% | Only Long Accuracy: 51.98% | Highest Validation Accuracy: 0.00%
Year: 2017 | Validation Accuracy: 54.18% | Only Long Accuracy: 56.97% | Highest Validation Accuracy: 0.00%
Training Years: 2006~2015 | Training Accuracy: 92.01% | Training OL Accuracy: 54.35%
Year: 2016~2017 Validation Accuracy: 54.87% | Only Long Accuracy: 54.47% | Highest Validation Accuracy: 0.00%
Year: 2018 | Test Accuracy: 55.38% | Only Long Accuracy: 52.59%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:True_reset:False_scale_card_classification_run4/reset:False_valYear:[2016, 2017]_testYear:2018.pt
Yea

 81%|████████  | 17/21 [57:29<13:34, 203.65s/it]

Training Years: 2006~2015 | Training Accuracy: 93.40% | Training OL Accuracy: 54.35%
Year: 2016~2017 Validation Accuracy: 53.68% | Only Long Accuracy: 54.47% | Highest Validation Accuracy: 57.46%
Year: 2018 | Test Accuracy: 48.61% | Only Long Accuracy: 52.59%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2016~2017: 57.46% | Highest Test Accuracy in 2018: 48.61%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:True_reset:False_scale_card_classification_run4/reset:False_valYear:[2016, 2017]_testYear:2018.pt
Model Validation Accuracy: 57.46% | Test Accuracy: 51.39%


<ipython-input-16-0813c403f401>:294: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Year: 2017 | Validation Accuracy: 51.00% | Only Long Accuracy: 56.97% | Highest Validation Accuracy: 0.00%
Year: 2018 | Validation Accuracy: 56.18% | Only Long Accuracy: 52.59% | Highest Validation Accuracy: 0.00%
Training Years: 2007~2016 | Training Accuracy: 89.52% | Training OL Accuracy: 53.93%
Year: 2017~2018 Validation Accuracy: 53.59% | Only Long Accuracy: 54.78% | Highest Validation Accuracy: 0.00%
Year: 2019 | Test Accuracy: 47.22% | Only Long Accuracy: 59.52%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:True_reset:False_scale_card_classification_run4/reset:False_valYear:[2017, 2018]_testYear:2019.pt
Year: 2017 | Validation Accuracy: 54.58% | Only Long Accuracy: 56.97% | Highest Validation Accuracy: 53.59%
Year: 2018 | Validation Accuracy: 56.57% | Only Long Accuracy: 52.59% | Highest Validation Accuracy: 53.59%
Training Years: 2007~2016 | Training Accuracy: 87.09% |

In [ ]:
# Scale = True
trainer.run()

Training model: scale_card_classification_run2



  0%|          | 0/21 [00:00<?, ?it/s]

Year: 2000 | Validation Accuracy: 52.78% | Only Long Accuracy: 47.62% | Highest Validation Accuracy: 0.00%
Year: 2001 | Validation Accuracy: 48.79% | Only Long Accuracy: 47.98% | Highest Validation Accuracy: 0.00%
Training Years: 1990~1999 | Training Accuracy: 53.48% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 50.80% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 0.00%
Year: 2002 | Test Accuracy: 48.41% | Only Long Accuracy: 44.44%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:False_reset:False_scale_card_classification_run2/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Year: 2000 | Validation Accuracy: 56.75% | Only Long Accuracy: 47.62% | Highest Validation Accuracy: 50.80%
Year: 2001 | Validation Accuracy: 51.21% | Only Long Accuracy: 47.98% | Highest Validation Accuracy: 50.80%
Training Years: 1990~1999 | Training Accuracy: 55.85% 

  5%|▍         | 1/21 [03:13<1:04:27, 193.36s/it]

Training Years: 1990~1999 | Training Accuracy: 97.39% | Training OL Accuracy: 53.60%
Year: 2000~2001 Validation Accuracy: 93.60% | Only Long Accuracy: 47.80% | Highest Validation Accuracy: 93.80%
Year: 2002 | Test Accuracy: 94.44% | Only Long Accuracy: 44.44%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2000~2001: 93.80% | Highest Test Accuracy in 2002: 94.44%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:False_reset:False_scale_card_classification_run2/reset:False_valYear:[2000, 2001]_testYear:2002.pt
Model Validation Accuracy: 93.80% | Test Accuracy: 88.10%


...

Training Years: 1991~2000 | Training Accuracy: 99.05% | Training OL Accuracy: 53.03%
Year: 2001~2002 Validation Accuracy: 91.60% | Only Long Accuracy: 46.20% | Highest Validation Accuracy: 96.00%
Year: 2003 | Test Accuracy: 91.27% | Only Long Accuracy: 54.37%
------------------------------------------------
***********************************************************
Highest Validation Accuracy in 2001~2002: 96.00% | Highest Test Accuracy in 2003: 91.27%
***********************************************************
=============================New Training Set====================================
Model loaded from /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:False_reset:False_scale_card_classification_run2/reset:False_valYear:[2001, 2002]_testYear:2003.pt
Model Validation Accuracy: 96.00% | Test Accuracy: 90.87%


<ipython-input-16-75b3ee47f388>:294: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(self.device))


Year: 2002 | Validation Accuracy: 95.63% | Only Long Accuracy: 44.44% | Highest Validation Accuracy: 0.00%
Year: 2003 | Validation Accuracy: 89.29% | Only Long Accuracy: 54.37% | Highest Validation Accuracy: 0.00%
Training Years: 1992~2001 | Training Accuracy: 98.45% | Training OL Accuracy: 52.93%
Year: 2002~2003 Validation Accuracy: 92.46% | Only Long Accuracy: 49.40% | Highest Validation Accuracy: 0.00%
Year: 2004 | Test Accuracy: 86.51% | Only Long Accuracy: 55.56%
------------------------------------------------
Model Saved at /content/drive/MyDrive/code/fintransformer/models/card_rolling/[^SPX]_valYrs:2_testYrs1_scale:False_reset:False_scale_card_classification_run2/reset:False_valYear:[2002, 2003]_testYear:2004.pt
Year: 2002 | Validation Accuracy: 96.43% | Only Long Accuracy: 44.44% | Highest Validation Accuracy: 92.46%
Year: 2003 | Validation Accuracy: 92.06% | Only Long Accuracy: 54.37% | Highest Validation Accuracy: 92.46%
Training Years: 1992~2001 | Training Accuracy: 95.92% 

 10%|▉         | 2/21 [08:43<1:22:54, 261.83s/it]


KeyboardInterrupt: 